# Profiling FLASH

In [1]:
import datetime
import logging
import os
import pathlib
import pprint
import shutil
import subprocess
import sys
import typing as t

import git
import pandas as pd

In [2]:
_LOG = logging.getLogger('profile_flash')

In [3]:
try:
    _HERE = pathlib.Path(__file__).parent.resolve()
except NameError:
    _HERE = pathlib.Path(os.getcwd()).resolve()
_HERE

PosixPath('/nfs2/mbysiek/Projects/docker-transpyle-flash')

In [4]:
_RESULTS_ROOT = pathlib.Path(_HERE, 'results')

In [5]:
_CACHED = {}

In [6]:
_CACHED

{}

check if the envvars are properly set

In [7]:
os.environ['CPATH']

'/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/papi-5.6.0-5p5d3qi2imn6m3szyl47gj73dpixhxpf/include:/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/hpctoolkit-master-wxx6jn7xuoo7lzqhg7kwxx4xl2nrlooc/include:/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/amrex-18.08.1-crdvjfdihxbjlam4swf2jzopgjv3dopv/include:/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/hypre-2.14.0-vea6lmriirgjzt4o6j4zxw2xbunynfs6/include:/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/openblas-0.3.2-femnsy3tmxlxehbgrk4yd4gmig2zjsle/include:/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/hdf5-1.8.20-p357p5eclyvy4vpqaownurdkn3mxv56v/include:/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/zlib-1.2.11-z7cqiqbuz4k4x4ideyvz3syxata5g5gd/include:/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/numactl-2.0.11-nawuf6bcuc

In [8]:
os.environ['LD_LIBRARY_PATH']

'/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/papi-5.6.0-5p5d3qi2imn6m3szyl47gj73dpixhxpf/lib:/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/hpctoolkit-master-wxx6jn7xuoo7lzqhg7kwxx4xl2nrlooc/lib:/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/amrex-18.08.1-crdvjfdihxbjlam4swf2jzopgjv3dopv/lib:/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/hypre-2.14.0-vea6lmriirgjzt4o6j4zxw2xbunynfs6/lib:/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/openblas-0.3.2-femnsy3tmxlxehbgrk4yd4gmig2zjsle/lib:/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/hdf5-1.8.20-p357p5eclyvy4vpqaownurdkn3mxv56v/lib:/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/zlib-1.2.11-z7cqiqbuz4k4x4ideyvz3syxata5g5gd/lib:/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/numactl-2.0.11-nawuf6bcuc5wuk7t5oncoltovb532qba/lib:/

In [9]:
shutil.which('hpcrun')

'/nfs2/mbysiek/Software/Spack/opt/spack/linux-ubuntu14.04-x86_64/gcc-8.2.0/hpctoolkit-master-wxx6jn7xuoo7lzqhg7kwxx4xl2nrlooc/bin/hpcrun'

runners implementation

In [10]:
def _run_and_check(cmd: str, wd: pathlib.Path, *,
                   test_name: str, phase_name: str):
    _LOG.warning('%s.%s: running "%s" with wd="%s"', test_name, phase_name, cmd, wd)
    # _dir = os.getcwd()
    # os.chdir(str(wd))
    cmd_result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE,
                                shell=True, cwd=str(wd))
    # os.chdir(_dir)
    log_dir = logs_path(test_name=test_name)
    log_dir.mkdir(parents=True, exist_ok=True)
    cmd_stdout = cmd_result.stdout.decode()
    with open(str(pathlib.Path(log_dir, '{}_stdout.log'.format(phase_name))),
              'w') as cmd_stdout_file:
        cmd_stdout_file.write(cmd_stdout)
    cmd_stderr = cmd_result.stderr.decode()
    with open(str(pathlib.Path(log_dir, '{}_stderr.log'.format(phase_name))),
              'w') as cmd_stderr_file:
        cmd_stderr_file.write(cmd_stderr)
    cmd_msg = None
    if cmd_result.returncode != 0:
        cmd_msg = '"{}" failed, returncode={}, logs were written to "{}"' \
            ' and last 50 lines of stderr follow:\n{}'.format(
                cmd, cmd_result.returncode, log_dir,
                ''.join(cmd_stderr.splitlines(keepends=True)[-50:]))
    assert cmd_result.returncode == 0, cmd_msg

In [11]:
def setup_flash(experiment, setup_dir, *,
                test_name: str, phase_name: str = 'setup'):
    _run_and_check('./setup -site spack ' + experiment, setup_dir,
                   test_name=test_name, phase_name=phase_name)


def make_flash(build_dir, *,
                test_name: str, phase_name: str = 'make'):
    _run_and_check('make', build_dir,
                   test_name=test_name, phase_name=phase_name)


def clean_flash(build_dir, *,
                test_name: str, phase_name: str = 'clean'):
    _run_and_check('make clean', build_dir,
                   test_name=test_name, phase_name=phase_name)

In [12]:
def hpctoolkit_profile(executable: pathlib.Path, results_path: pathlib.Path, sample_size: int, mpi_proc: int = 0, *,
                       test_name: str, phase_name: str = 'profile'):
    hpcrun = 'hpcrun' # shutil.which('hpcrun')
    results_path.parent.mkdir(exist_ok=True)
    hpcrun_command = '{} -o "{}" {}'.format(hpcrun, results_path, executable)
    if with_mpi > 0:
        hpcrun_command = 'mpirun -np {} {}'.format(mpi_proc, hpcrun_command)
    _LOG.warning('%s.%s: running the experiment %i times...', test_name, phase_name, sample_size)
    for i in range(sample_size):
        _run_and_check(hpcrun_command, executable.parent,
                       test_name=test_name, phase_name=phase_name)
        # subprocess.run(hpcrun_command, shell=True)

In [13]:
def hpctoolkit_summarize(executable: pathlib.Path, results_path: pathlib.Path, source_path: pathlib.Path, *,
                         test_name: str, phase_name: str = 'summarize'):
    hpcstruct = 'hpcstruct'  # shutil.which('hpcstruct')
    hpcprof = 'hpcprof' # shutil.which('hpcprof')
    #source_path.joinpath('*')
    struct_path = results_path.joinpath(executable.name + '.hpcstruct')
    hpcstruct_command = '{} --verbose -I "{}" -o {} {}'.format(
        hpcstruct, './*', struct_path, executable)
    _run_and_check(hpcstruct_command, source_path,
                   test_name=test_name, phase_name='{}.hpcstruct'.format(phase_name))
    # hpcprof -I "${source}" "${results_path}" \
    #   -S "${results_path}/flash4.hpcstruct" \
    #   -M stats \
    #   -o "${results_path}_db"
    # source_path.joinpath('+')
    hpcprof_command = '{} --verbose -I "{}" {} -S {} -M stats -o {}'.format(
        hpcprof, './+', results_path, struct_path, profile_db_path(test_name=test_name))
    _run_and_check(hpcprof_command, source_path,
                   test_name=test_name, phase_name='{}.hpcprof'.format(phase_name))

In [14]:
def profile_flash(app_name: str, executable: pathlib.Path, source_path: pathlib.Path, sample_size: int, *,
                  test_name: str):
    results_path = profile_path(test_name=test_name)
    hpctoolkit_profile(executable, results_path, sample_size, test_name=test_name)
    hpctoolkit_summarize(executable, results_path, source_path, test_name=test_name)

In [15]:
def profile_experiment(app_name: str, experiment: str, branch: str, sample_size: int, *,
                       test_name: str):
    app_dir = pathlib.Path(_HERE, app_name)
    setup_dir = {
        'flash-subset': app_dir.joinpath('FLASH4.4')
        }.get(app_name, app_dir)
    repo = git.Repo(str(app_dir))
    assert not repo.is_dirty(untracked_files=True), repo
    if str(repo.active_branch) != branch:
        _LOG.warning('%s: checking out %s', test_name, branch)
        repo.git.checkout(branch)
    build_dir = setup_dir.joinpath('object')
    executable = build_dir.joinpath('flash4')
    setup_flash(experiment, setup_dir,
                test_name=test_name)
    make_flash(build_dir,
               test_name=test_name)
    profile_flash(app_name, executable, app_dir, sample_size,
                  test_name=test_name)
    clean_flash(build_dir,
                test_name=test_name)

In [16]:
def date_str(date=None) -> str:
    if date is None:
        date = _NOW
    return date.strftime('%Y%m%d-%H%M%S')


def logs_path(date=None, *, test_name) -> pathlib.Path:
    return _RESULTS_ROOT.joinpath('{}_{}'.format(date_str(date), test_name))


def profile_path(date=None, *, test_name) -> pathlib.Path:
    return _RESULTS_ROOT.joinpath('profile_{}_{}'.format(date_str(date), test_name))


def profile_db_path(date=None, *, test_name) -> pathlib.Path:
    profile_dir = profile_path(date, test_name=test_name)
    return profile_dir.with_name(profile_dir.name + '_db')

## Data analysis

In [17]:
import hpctoolkit_dataframe
from hpctoolkit_dataframe import HPCtoolkitDataFrame

In [95]:
%%time

_profile_path = profile_db_path(
    datetime.datetime(2018, 9, 5, 12, 5, 19, 934255), test_name='subset_Sedov_base_a').joinpath('experiment.xml')
df = HPCtoolkitDataFrame(path=_profile_path, max_depth=25)

CPU times: user 11.1 s, sys: 24 ms, total: 11.1 s
Wall time: 11.1 s


In [20]:
%%time

import line_profiler
profiler = line_profiler.LineProfiler()
profiler.add_module(hpctoolkit_dataframe)
profiler.enable()

_profile_path = profile_db_path(
    datetime.datetime(2018, 9, 5, 12, 5, 19, 934255), test_name='subset_Sedov_base_a').joinpath('experiment.xml')
df = HPCtoolkitDataFrame(path=_profile_path, max_depth=None)

profiler.disable()
profiler.print_stats()

ValueError: can only convert an array of size 1 to a Python scalar

In [23]:
df.hot_path(threshold=0.5)

,CPUTIME (usec):Sum (I),CPUTIME (usec):Mean (I),CPUTIME (usec):Mean (I) %,CPUTIME (usec):Mean:num-src (I),CPUTIME (usec):StdDev (I),CPUTIME (usec):StdDev:accum2 (I),CPUTIME (usec):StdDev:num-src (I),CPUTIME (usec):CfVar (I),CPUTIME (usec):CfVar:accum2 (I),CPUTIME (usec):CfVar:num-src (I),...,CPUTIME (usec):Mean:num-src (E),CPUTIME (usec):StdDev (E),CPUTIME (usec):StdDev:accum2 (E),CPUTIME (usec):StdDev:num-src (E),CPUTIME (usec):CfVar (E),CPUTIME (usec):CfVar:accum2 (E),CPUTIME (usec):CfVar:num-src (E),CPUTIME (usec):Min (E),CPUTIME (usec):Max (E),location
(),977781000.0,19555620.0,1.000000,50.0,133140.585848,1.912200e+16,50.0,0.006808,1.912200e+16,50.0,...,50.0,133140.585848,NaN,50.0,NaN,NaN,50.0,1.930300e+07,19867400.0,()
"('<program root>',)",977141000.0,19542820.0,0.999345,50.0,134857.137742,1.909700e+16,50.0,0.006901,1.909700e+16,50.0,...,50.0,NaN,NaN,50.0,NaN,NaN,50.0,2.225070e-308,NaN,"(<program root>,)"
"('<program root>', '<callsite 13385.7>')",977141000.0,19542820.0,1.000000,50.0,134857.137742,1.909700e+16,50.0,0.006901,1.909700e+16,50.0,...,50.0,NaN,NaN,50.0,NaN,NaN,50.0,2.225070e-308,NaN,"(<program root>, <callsite 13385.7>)"
"('<program root>', '<callsite 13385.7>', 'main')",977141000.0,19542820.0,1.000000,50.0,134857.137742,1.909700e+16,50.0,0.006901,1.909700e+16,50.0,...,50.0,NaN,NaN,50.0,NaN,NaN,50.0,2.225070e-308,NaN,"(<program root>, <callsite 13385.7>, main)"
"('<program root>', '<callsite 13385.7>', 'main', '<callsite 337.1003>')",960942000.0,19218840.0,0.983422,50.0,127236.214970,1.846900e+16,50.0,0.006620,1.846900e+16,50.0,...,50.0,NaN,NaN,50.0,NaN,NaN,50.0,2.225070e-308,NaN,"(<program root>, <callsite 13385.7>, main, <ca..."
"('<program root>', '<callsite 13385.7>', 'main', '<callsite 337.1003>', 'driver_evolveflash_')",960942000.0,19218840.0,1.000000,50.0,127236.214970,1.846900e+16,50.0,0.006620,1.846900e+16,50.0,...,50.0,1112.579919,6.315480e+07,50.0,6.999999,6.315480e+07,50.0,7.947000e+03,7947.0,"(<program root>, <callsite 13385.7>, main, <ca..."
"('<program root>', '<callsite 13385.7>', 'main', '<callsite 337.1003>', 'driver_evolveflash_', '<loop 460>')",959013000.0,19180260.0,0.997993,50.0,125005.329486,1.839490e+16,50.0,0.006517,1.839490e+16,50.0,...,50.0,1112.579919,6.315480e+07,50.0,6.999999,6.315480e+07,50.0,7.947000e+03,7947.0,"(<program root>, <callsite 13385.7>, main, <ca..."
"('<program root>', '<callsite 13385.7>', 'main', '<callsite 337.1003>', 'driver_evolveflash_', '<loop 460>', '<callsite 461.4265>')",822330000.0,16446600.0,0.857475,50.0,115535.449105,1.352520e+16,50.0,0.007025,1.352520e+16,50.0,...,50.0,NaN,NaN,50.0,NaN,NaN,50.0,2.225070e-308,NaN,"(<program root>, <callsite 13385.7>, main, <ca..."
"('<program root>', '<callsite 13385.7>', 'main', '<callsite 337.1003>', 'driver_evolveflash_', '<loop 460>', '<callsite 461.4265>', 'hydro_')",822330000.0,16446600.0,1.000000,50.0,115535.449105,1.352520e+16,50.0,0.007025,1.352520e+16,50.0,...,50.0,NaN,NaN,50.0,NaN,NaN,50.0,2.225070e-308,NaN,"(<program root>, <callsite 13385.7>, main, <ca..."
"('<program root>', '<callsite 13385.7>', 'main', '<callsite 337.1003>', 'driver_evolveflash_', '<loop 460>', '<callsite 461.4265>', 'hydro_', '<callsite 1705.5234>')",633651000.0,12673020.0,0.770556,50.0,83451.061108,8.030620e+15,50.0,0.006585,8.030620e+15,50.0,...,50.0,NaN,NaN,50.0,NaN,NaN,50.0,2.225070e-308,NaN,"(<program root>, <callsite 13385.7>, main, <ca..."


In [121]:
len(df)

12

In [122]:
df.compact[:10]

,CPUTIME (usec):Mean (I),CPUTIME (usec):Mean (I) %,CPUTIME (usec):Mean (E),CPUTIME (usec):Mean (E) %
(),19555620.00,1.000000,19555620.00,1.000000
"('<partial call paths>',)",12809.54,0.000655,NaN,NaN
"('<partial call paths>', '<callsite 13379>')",12809.54,1.000000,NaN,NaN
"('<partial call paths>', '<callsite 13379>', '<unknown procedure>')",12809.54,1.000000,12809.54,0.000655
"('<partial call paths>', '<callsite 13379>', '<unknown procedure>', '<statement 13381>')",12809.54,1.000000,12809.54,0.000655
"('<program root>',)",19542820.00,0.999345,NaN,NaN
"('<program root>', '<callsite 13385>')",19542820.00,1.000000,NaN,NaN
"('<program root>', '<callsite 13385>', 'main')",19542820.00,1.000000,NaN,NaN
"('<program root>', '<callsite 13385>', 'main', '<callsite 337>')",6949.88,0.000356,NaN,NaN
"('<program root>', '<callsite 13385>', 'main', '<callsite 337>', 'driver_initflash_')",317018.00,0.016222,NaN,NaN


In [92]:
df.basic_e#[::len(df) // 4]

,CPUTIME (usec):Sum (E),CPUTIME (usec):Mean (E),CPUTIME (usec):Mean (E) %,CPUTIME (usec):Min (E),CPUTIME (usec):Max (E),CPUTIME (usec):StdDev (E)
(),NaN,NaN,NaN,2.225070e-308,NaN,NaN
"('<partial call paths>',)",NaN,NaN,NaN,2.225070e-308,NaN,NaN
"('<partial call paths>', '<callsite 13379>')",NaN,NaN,NaN,2.225070e-308,NaN,NaN
"('<partial call paths>', '<callsite 13379>', '<unknown procedure>')",640477.0,12809.54,NaN,7.957000e+03,47896.0,10356.528617
"('<partial call paths>', '<callsite 13379>', '<unknown procedure>', '<statement 13381>')",640477.0,12809.54,NaN,7.957000e+03,47896.0,10356.528617
"('<program root>',)",NaN,NaN,NaN,2.225070e-308,NaN,NaN
"('<program root>', '<callsite 13385>')",NaN,NaN,NaN,2.225070e-308,NaN,NaN
"('<program root>', '<callsite 13385>', 'main')",NaN,NaN,NaN,2.225070e-308,NaN,NaN
"('<program root>', '<callsite 13385>', 'main', '<callsite 337>')",NaN,NaN,NaN,2.225070e-308,NaN,NaN
"('<program root>', '<callsite 13385>', 'main', '<callsite 337>', 'driver_initflash_')",NaN,NaN,NaN,2.225070e-308,NaN,NaN


In [34]:
df[['CPUTIME (usec):Sum (I)', 'CPUTIME (usec):Sum (E)']][::len(df) // 4]

,CPUTIME (usec):Sum (I),CPUTIME (usec):Sum (E)
(),977781000.0,NaN
"('<partial call paths>', '<callsite 13379>', '<unknown procedure>')",640477.0,640477.0
"('<program root>', '<callsite 13385>')",977141000.0,NaN
"('<program root>', '<callsite 13385>', 'main', '<callsite 337>', 'driver_initflash_')",15850900.0,NaN


In [21]:
df[::len(df) // 4]

,CPUTIME (usec):Sum (I),CPUTIME (usec):Mean (I),CPUTIME (usec):Mean (I) %,CPUTIME (usec):Mean:num-src (I),CPUTIME (usec):StdDev (I),CPUTIME (usec):StdDev:accum2 (I),CPUTIME (usec):StdDev:num-src (I),CPUTIME (usec):CfVar (I),CPUTIME (usec):CfVar:accum2 (I),CPUTIME (usec):CfVar:num-src (I),...,CPUTIME (usec):Mean:num-src (E),CPUTIME (usec):StdDev (E),CPUTIME (usec):StdDev:accum2 (E),CPUTIME (usec):StdDev:num-src (E),CPUTIME (usec):CfVar (E),CPUTIME (usec):CfVar:accum2 (E),CPUTIME (usec):CfVar:num-src (E),CPUTIME (usec):Min (E),CPUTIME (usec):Max (E),location
(),977781000.0,19555620.00,1.000000,50.0,133140.585848,1.912200e+16,50.0,0.006808,1.912200e+16,50.0,...,50.0,NaN,NaN,50.0,NaN,NaN,50.0,2.225070e-308,NaN,()
"('<partial call paths>',)",640477.0,12809.54,0.000655,50.0,10356.528617,1.356710e+10,50.0,0.808501,1.356710e+10,50.0,...,50.0,NaN,NaN,50.0,NaN,NaN,50.0,2.225070e-308,NaN,"(<partial call paths>,)"
"('<partial call paths>', '<callsite 13379>')",640477.0,12809.54,1.000000,50.0,10356.528617,1.356710e+10,50.0,0.808501,1.356710e+10,50.0,...,50.0,NaN,NaN,50.0,NaN,NaN,50.0,2.225070e-308,NaN,"(<partial call paths>, <callsite 13379>)"
"('<partial call paths>', '<callsite 13379>', '<unknown procedure>')",640477.0,12809.54,1.000000,50.0,10356.528617,1.356710e+10,50.0,0.808501,1.356710e+10,50.0,...,50.0,10356.528617,1.356710e+10,50.0,0.808501,1.356710e+10,50.0,7.957000e+03,47896.0,"(<partial call paths>, <callsite 13379>, <unkn..."
"('<program root>',)",977141000.0,19542820.00,0.999345,50.0,134857.137742,1.909700e+16,50.0,0.006901,1.909700e+16,50.0,...,50.0,NaN,NaN,50.0,NaN,NaN,50.0,2.225070e-308,NaN,"(<program root>,)"
"('<program root>', '<callsite 13385>')",977141000.0,19542820.00,1.000000,50.0,134857.137742,1.909700e+16,50.0,0.006901,1.909700e+16,50.0,...,50.0,NaN,NaN,50.0,NaN,NaN,50.0,2.225070e-308,NaN,"(<program root>, <callsite 13385>)"
"('<program root>', '<callsite 13385>', 'main')",977141000.0,19542820.00,1.000000,50.0,134857.137742,1.909700e+16,50.0,0.006901,1.909700e+16,50.0,...,50.0,NaN,NaN,50.0,NaN,NaN,50.0,2.225070e-308,NaN,"(<program root>, <callsite 13385>, main)"


In [58]:
location = ('<program root>',)

def location_filter(series: pd.Series, location: tuple):
    series_location = series['location']#.item()
    #if series_location == ():
    #    return series
    #print(series_location)
    if len(series_location) < len(location):
        return False
    location_prefix = series_location[:len(location)]
    if location_prefix != location:
        return False
    return True

mask = df.apply(location_filter, axis=1, args=(location,))

masked = df[mask]

index = masked['CPUTIME (usec):Mean (I) %'].argmax()

masked.loc[index]['location']

/homes/mbysiek/.local/lib/python3.6/site-packages/ipykernel_launcher.py:19: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.


('<program root>',
 '<callsite 13385>',
 'main',
 '<callsite 337>',
 'driver_evolveflash_')

In [190]:
df.compact.filter(like='hy_upwindtransverseflux_\')', axis=0)

,CPUTIME (usec):Sum (I),CPUTIME (usec):Mean (I),CPUTIME (usec):Min (I),CPUTIME (usec):Max (I)
"('<program root>', '<callsite 13385>', 'main', '<callsite 337>', 'driver_evolveflash_', '<loop 460>', '<callsite 461>', 'hydro_', '<callsite 1705>', 'hy_advance_', '<loop 6622>', '<callsite 6623>', 'hy_computefluxes_', '<callsite 6633>', 'hy_getriemannstate_', '<loop 6882>', '<loop 6884>', '<callsite 6885>', 'hy_datareconstonestep_', '<loop 6698>', '<callsite 6699>', 'hy_datareconstructnormaldir_ppm_', '<callsite 6141>', 'hy_upwindtransverseflux_')",58288000.0,1165760.0,985530.0,1368360.0


In [200]:
df[df.location == ()]['CPUTIME (usec):Mean (I)'].item()

19555620.0

In [207]:
for value, aa in df[['CPUTIME (usec):Mean (I)', 'location']][:10].items():
    print(value, aa)

CPUTIME (usec):Mean (I) ()                                                                                          19555620.00
('<partial call paths>',)                                                                      12809.54
('<partial call paths>', '<callsite 13379>')                                                   12809.54
('<partial call paths>', '<callsite 13379>', '<unknown procedure>')                            12809.54
('<partial call paths>', '<callsite 13379>', '<unknown procedure>', '<statement 13381>')       12809.54
('<program root>',)                                                                         19542820.00
('<program root>', '<callsite 13385>')                                                      19542820.00
('<program root>', '<callsite 13385>', 'main')                                              19542820.00
('<program root>', '<callsite 13385>', 'main', '<callsite 337>')                                6949.88
('<program root>', '<callsite 13385>', '

run the experiments

In [60]:
_NOW = datetime.datetime.now()

In [82]:
_NOW

datetime.datetime(2018, 9, 5, 12, 5, 19, 934255)

In [54]:
profile_experiment('flash-subset', 'Sedov -auto -2d +Mode1', 'transpyle_experiments_outlined', 50,
                   test_name='subset_sedov_outlined_a')

subset_sedov_outlined_a.profile: running the experiment 1 times...
subset_sedov_outlined_a.profile: running "hpcrun -o "/nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-114027_subset_sedov_outlined_a" /nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object/flash4" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_sedov_outlined_a.summarize.hpcstruct: running "hpcstruct --verbose -I "./*" -o /nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-114027_subset_sedov_outlined_a/flash4.hpcstruct /nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object/flash4" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset"
subset_sedov_outlined_a.summarize.hpcprof: running "hpcprof --verbose -I "./+" /nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-114027_subset_sedov_outlined_a -S /nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_201809

In [ ]:
clean_flash(pathlib.Path(_HERE, 'flash-subset', 'FLASH4.4', 'object'), 'subset_sedov_outlined_a')

In [54]:
profile_experiment('flash-subset', 'Sod -auto -2d +Mode1', 'transpyle_experiments_outlined', 50,
                   test_name='subset_sedov_outlined_a')

subset_sedov_outlined_a.profile: running the experiment 1 times...
subset_sedov_outlined_a.profile: running "hpcrun -o "/nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-114027_subset_sedov_outlined_a" /nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object/flash4" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_sedov_outlined_a.summarize.hpcstruct: running "hpcstruct --verbose -I "./*" -o /nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-114027_subset_sedov_outlined_a/flash4.hpcstruct /nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object/flash4" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset"
subset_sedov_outlined_a.summarize.hpcprof: running "hpcprof --verbose -I "./+" /nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-114027_subset_sedov_outlined_a -S /nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_201809

In [ ]:
profile_experiment('flash-subset', 'Sedov -auto -2d +Mode1', 'transpyle_experiments_autoinlined', 50,
                   test_name='subset_sedov_autoinlined_a')

In [ ]:
profile_experiment('flash-subset', 'Sod -auto -2d +Mode1', 'transpyle_experiments_autoinlined', 50,
                   test_name='subset_sedov_autoinlined_a')

In [61]:
for branch, nickname in {
        'transpyle_experiments': 'base', 'transpyle_experiments_outlined': 'outlined',
        'transpyle_experiments_autoinlined': 'autoinlined'}.items():
    for problem in ['Sod', 'Sedov']:
        test_name = 'subset_{}_{}_b'.format(problem, nickname)
        profile_experiment('flash-subset', '{} -auto -2d +Mode1'.format(problem), branch, 50,
                           test_name=test_name)

subset_Sod_base_a.setup: running "./setup -site spack Sod -auto -2d +Mode1" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4"
subset_Sod_base_a.make: running "make" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_Sod_base_a.profile: running the experiment 50 times...
subset_Sod_base_a.profile: running "hpcrun -o "/nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-120519_subset_Sod_base_a" /nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object/flash4" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_Sod_base_a.profile: running "hpcrun -o "/nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-120519_subset_Sod_base_a" /nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object/flash4" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_Sod_base_a.profile: running "

subset_Sod_base_a.profile: running "hpcrun -o "/nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-120519_subset_Sod_base_a" /nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object/flash4" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_Sod_base_a.profile: running "hpcrun -o "/nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-120519_subset_Sod_base_a" /nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object/flash4" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_Sod_base_a.profile: running "hpcrun -o "/nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-120519_subset_Sod_base_a" /nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object/flash4" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_Sod_base_a.profile: running "hpcrun -o "/nfs2/mbysiek/Pro

subset_Sod_base_a.clean: running "make clean" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_Sedov_base_a.setup: running "./setup -site spack Sedov -auto -2d +Mode1" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4"
subset_Sedov_base_a.make: running "make" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_Sedov_base_a.profile: running the experiment 50 times...
subset_Sedov_base_a.profile: running "hpcrun -o "/nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-120519_subset_Sedov_base_a" /nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object/flash4" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_Sedov_base_a.profile: running "hpcrun -o "/nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-120519_subset_Sedov_base_a" /nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subse

subset_Sedov_base_a.profile: running "hpcrun -o "/nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-120519_subset_Sedov_base_a" /nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object/flash4" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_Sedov_base_a.profile: running "hpcrun -o "/nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-120519_subset_Sedov_base_a" /nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object/flash4" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_Sedov_base_a.profile: running "hpcrun -o "/nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-120519_subset_Sedov_base_a" /nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object/flash4" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_Sedov_base_a.profile: running "hpcrun -o "/nf

subset_Sedov_base_a.summarize.hpcprof: running "hpcprof --verbose -I "./+" /nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-120519_subset_Sedov_base_a -S /nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-120519_subset_Sedov_base_a/flash4.hpcstruct -M stats -o /nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-120519_subset_Sedov_base_a_db" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset"
subset_Sedov_base_a.clean: running "make clean" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_Sod_outlined_a: checking out transpyle_experiments_outlined
subset_Sod_outlined_a.setup: running "./setup -site spack Sod -auto -2d +Mode1" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4"
subset_Sod_outlined_a.make: running "make" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_Sod_outlined_a.profile: running t

...

subset_Sedov_outlined_a.profile: running "hpcrun -o "/nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-120519_subset_Sedov_outlined_a" /nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object/flash4" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_Sedov_outlined_a.profile: running "hpcrun -o "/nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-120519_subset_Sedov_outlined_a" /nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object/flash4" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_Sedov_outlined_a.profile: running "hpcrun -o "/nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-120519_subset_Sedov_outlined_a" /nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object/flash4" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_Sedov_outlined_a.summ

subset_Sod_autoinlined_a.profile: running "hpcrun -o "/nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-120519_subset_Sod_autoinlined_a" /nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object/flash4" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_Sod_autoinlined_a.profile: running "hpcrun -o "/nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-120519_subset_Sod_autoinlined_a" /nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object/flash4" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_Sod_autoinlined_a.profile: running "hpcrun -o "/nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-120519_subset_Sod_autoinlined_a" /nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object/flash4" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_Sod_autoinlined

subset_Sod_autoinlined_a.profile: running "hpcrun -o "/nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-120519_subset_Sod_autoinlined_a" /nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object/flash4" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_Sod_autoinlined_a.profile: running "hpcrun -o "/nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-120519_subset_Sod_autoinlined_a" /nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object/flash4" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_Sod_autoinlined_a.profile: running "hpcrun -o "/nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-120519_subset_Sod_autoinlined_a" /nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object/flash4" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_Sod_autoinlined

subset_Sedov_autoinlined_a.profile: running "hpcrun -o "/nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-120519_subset_Sedov_autoinlined_a" /nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object/flash4" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_Sedov_autoinlined_a.profile: running "hpcrun -o "/nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-120519_subset_Sedov_autoinlined_a" /nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object/flash4" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_Sedov_autoinlined_a.profile: running "hpcrun -o "/nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-120519_subset_Sedov_autoinlined_a" /nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object/flash4" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_Sed

subset_Sedov_autoinlined_a.profile: running "hpcrun -o "/nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-120519_subset_Sedov_autoinlined_a" /nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object/flash4" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_Sedov_autoinlined_a.profile: running "hpcrun -o "/nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-120519_subset_Sedov_autoinlined_a" /nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object/flash4" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_Sedov_autoinlined_a.profile: running "hpcrun -o "/nfs2/mbysiek/Projects/docker-transpyle-flash/results/profile_20180905-120519_subset_Sedov_autoinlined_a" /nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object/flash4" with wd="/nfs2/mbysiek/Projects/docker-transpyle-flash/flash-subset/FLASH4.4/object"
subset_Sed